# Data Analysis of Eye Tracking Data in Chest-Xrays



## Set Up

### Install requirements

In [ ]:
# install required dependencies
#%pip install -r requirements.txt

### SETUP GPU

In [ ]:
# setting up GPU
import tensorflow as tf
from util.test_gpu import cpu
from util.test_gpu import gpu
import timeit

print("Comparing CPU vs GPU executions...")
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('\n\nThis error most likely means that this notebook is not '
            'configured to use a GPU.  Change this in Notebook Settings via the '
            'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise SystemError('GPU device not found')
    
# We run each op once to warm up
#cpu()
#gpu()

# Run the op several times.
#print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
#    '(batch x height x width x channel). Sum of ten runs.')
#print('CPU (s):')

#cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
#print(cpu_time)
#print('GPU (s):')
#gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
#print(gpu_time)
#print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))


### Initializing Notebook

In [1]:
# general libs
import numpy as np
import pandas as pd
import pingouin as pt
import warnings

# visualisation
import IPython
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell # Enable multiple cell outputs
from tqdm.notebook import tqdm # IPython/Jupyter Notebook progressbar decorator for iterators.
import matplotlib.pyplot as plt
import seaborn as sns

# import my own functions
from  util.global_vars import GlobalVars
from util.aux_functions import *
from dataprocessing.patient import Patient
from dataprocessing.mimicdata import MIMICDataProcessor


C:\Users\cmore\AppData\Local\Temp\ipykernel_19272\1927748649.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# package configurations
# no display limit in dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore') # ignore warnings

sns.set() # sets seaborn as the main visualisationlib
sns.set_context('notebook') # Larger scale for plots in notebooks

# Plotting pretty figures and avoid blurry images
%config InlineBackend.figure_format = 'retina'
# will make your plot outputs appear and be stored within the notebook.
%matplotlib inline
plt.rcParams.update({'figure.max_open_warning': 1})

# Enable multiple cell outputs
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# initialize vars
# first argument asks if the program will run in Google Colab
# second argument asks if the user wants to pre=process the dataset (True) or
# if the user wants to load a previously process dataset from a dictionary file (False)
gv = GlobalVars( isColab = False, flag = False )

# if FLAG is true, process the dataset, otherwise load it from disk
create_XAMI_MIMIC_Dictionaries( gv ) if gv.getFLAG() else load_XAMI_MIMIC_Dictionaries( gv )

# for convinience, let's put the dictinaries in separate variables
PATIENTS_DIC, IMG_TO_DIAGNOSIS, IMG_TO_PATIENT = gv.getMIMICDictionaries()

print("Total patients loaded: " + str(len(PATIENTS_DIC)))
print("Total Images -> Diagnosis loaded: " + str(len(IMG_TO_DIAGNOSIS)))
print("Total Images -> Patient Keys loaded: " + str(len(IMG_TO_PATIENT)))

Loading dictionaries...
Total patients loaded: 1083
Total Images -> Diagnosis loaded: 1083
Total Images -> Patient Keys loaded: 1083


In [4]:
patientsBoth = MIMICDataProcessor( gv ).findPatientsInBothDatasets()
patientsBoth


FOUND PATIENT: 19565653_IMG1 with DICOM 060cf092-fe76bdf7-19fee515-26cbef2c-5c16ba6f


KeyError: 0

### DATA EXPLORATION OF GENERAL SCANPATHS, PUPIL DILATIONS, FIXATION DURATIONS

